<a href="https://colab.research.google.com/github/Mv2077/Anatomia_de_Grey-medical-specialty-text-classifier-/blob/main/Anatomia_de_Grey(2_0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Medical cases classification

In [ ]:
!pip install evaluate

In [ ]:
!pip install transformers==4.30.0 peft==0.5.0 --upgrade

In [ ]:
!pip install streamlit

In [ ]:
!pip install pyngrok

In [ ]:
!pip install numpy pandas

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4

In [ ]:
!git clone https://huggingface.co/datasets/hpe-ai/medical-cases-classification-tutorial



In [ ]:
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification
from transformers import DataCollatorWithPadding
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
import tensorflow as tf
from transformers import TrainingArguments
from transformers import Trainer
from transformers import EarlyStoppingCallback
import numpy as np
import os
os.environ["WANDB_DISABLED"] = "true"
from transformers import EarlyStoppingCallback
from torch.utils.data import DataLoader
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import streamlit as st
from sklearn.utils import resample
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from evaluate import load
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import classification_report
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoTokenizer

In [ ]:

train_url = "https://huggingface.co/datasets/hpe-ai/medical-cases-classification-tutorial/resolve/main/medical_cases_train.csv"
test_url = "https://huggingface.co/datasets/hpe-ai/medical-cases-classification-tutorial/resolve/main/medical_cases_test.csv"
validation_url = "https://huggingface.co/datasets/hpe-ai/medical-cases-classification-tutorial/resolve/main/medical_cases_validation.csv"

df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
df_validation = pd.read_csv(validation_url)

print(df_train.head())


##Business Understanding




In [ ]:
# Objective:
# Classify medical cases based on textual descriptions.
# This helps support medical decision-making or automate triage processes.

print("Goal: Predict the medical category of a case from its description.")
df_train.info()
df_train.head()


##Data understanding

In [ ]:
print(df_train.columns)


In [ ]:
print("Training Set Info:")
print(df_train.info())
print("\nSample Rows:")
print(df_train.sample(5))

print("\nMissing Values:")
print(df_train.isnull().sum())

print("\nClass Distribution:")
print(df_train['medical_specialty'].value_counts())

plt.figure(figsize=(10, 5))
sns.countplot(y='medical_specialty', data=df_train, order=df_train['medical_specialty'].value_counts().index, palette='viridis')
plt.title("Distribution of Medical Specialties in Training Set")
plt.xlabel("Count")
plt.ylabel("Medical Specialty")
plt.tight_layout()
plt.show()

df_train['text_length'] = df_train['transcription'].apply(len)

print("\nText Length Statistics:")
print(df_train['text_length'].describe())

plt.figure(figsize=(8, 4))
sns.histplot(df_train['text_length'], bins=30, kde=True, color='skyblue')
plt.title("Distribution of Transcription Lengths")
plt.xlabel("Number of Characters")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

##Data Preparation

In [ ]:
label_encoder = LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train['medical_specialty'])
df_test['label'] = label_encoder.transform(df_test['medical_specialty'])

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokens = text.split()
    tokens = [w for w in tokens if w not in stop_words]
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    return " ".join(tokens)


df_train['description_clean'] = df_train['description'].apply(clean_text)
df_test['description_clean'] = df_test['description'].apply(clean_text)


label_encoder = LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train['medical_specialty'])
df_test['label'] = label_encoder.transform(df_test['medical_specialty'])


print("\nExemplo de transcrição limpa:")
print(df_train['transcription_clean'].iloc[0])
print("\nLabel original:", df_train['medical_specialty'].iloc[0])
print("Label codificado:", df_train['label'].iloc[0])


print("\nDistribuição de labels (treino):")
print(df_train['medical_specialty'].value_counts())


In [ ]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return " ".join(tokens)


train_url = "https://huggingface.co/datasets/hpe-ai/medical-cases-classification-tutorial/resolve/main/medical_cases_train.csv"
test_url = "https://huggingface.co/datasets/hpe-ai/medical-cases-classification-tutorial/resolve/main/medical_cases_test.csv"
validation_url = "https://huggingface.co/datasets/hpe-ai/medical-cases-classification-tutorial/resolve/main/medical_cases_validation.csv"

df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
df_validation = pd.read_csv(validation_url)

df = pd.concat([df_train, df_test, df_validation], ignore_index=True)

df = df.dropna(subset=["description", "medical_specialty"])
df["description_clean"] = df["description"].apply(clean_text)

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["medical_specialty"])

print("\nDistribuição original das classes:")
print(df["medical_specialty"].value_counts())

X = df["description_clean"]
y = df["label"]

from imblearn.over_sampling import RandomOverSampler # Import RandomOverSampler
from sklearn.model_selection import train_test_split # Import train_test_split

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X.values.reshape(-1, 1), y)

df_balanced = pd.DataFrame({
    "description_clean": X_resampled.flatten(),
    "label": y_resampled
})


df_balanced["medical_specialty"] = label_encoder.inverse_transform(df_balanced["label"])

df_train, df_test = train_test_split(
    df_balanced,
    test_size=0.2,
    stratify=df_balanced["label"],
    random_state=42
)

print("\n✅ Split final realizado.")
print("Distribuição no treino:")
print(df_train["medical_specialty"].value_counts())

#Modeling

In [ ]:
#MODELING com Linear Regression

vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(df_train['description_clean'])
X_test = vectorizer.transform(df_test['description_clean'])

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, df_train['label'])

y_pred = clf.predict(X_test)
print(classification_report(df_test['label'], y_pred, target_names=label_encoder.classes_))

In [ ]:
joblib.dump(clf, 'logistic_regression_model.pkl')
print("Logistic Regression model saved as 'logistic_regression_model.pkl'")

In [ ]:
#Modelling com Naive Bayes

X_train, X_test, y_train, y_test = train_test_split(
    df_train["description_clean"],
    df_train["label"],
    test_size=0.1,
    stratify=df_train["label"],
    random_state=42
)

nb_pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1, 2), stop_words="english"),
    MultinomialNB()
)

nb_pipeline.fit(X_train, y_train)

y_pred = nb_pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


In [ ]:
joblib.dump(nb_pipeline, 'naive_bayes_pipeline.pkl')
print("Naive Bayes pipeline saved as 'naive_bayes_pipeline.pkl'")

In [ ]:
#Modeling com GPT-2

tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token


def tokenize_gpt2(example):
    return tokenizer_gpt2(
        example["description_clean"],
        truncation=True,
        padding="max_length",
        max_length=256
    )


train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_train["description_clean"].tolist(),
    df_train["label"].tolist(),
    test_size=0.1,
    random_state=42,
    stratify=df_train["label"]
)

train_dataset = Dataset.from_dict({
    "description_clean": train_texts,
    "label": train_labels
})
val_dataset = Dataset.from_dict({
    "description_clean": val_texts,
    "label": val_labels
})

train_dataset = train_dataset.map(tokenize_gpt2, batched=True)
val_dataset = val_dataset.map(tokenize_gpt2, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

config = GPT2Config.from_pretrained("gpt2", num_labels=df_train["label"].nunique())
config.pad_token_id = tokenizer_gpt2.pad_token_id
model_gpt2 = AutoModelForSequenceClassification.from_pretrained("gpt2", config=config)

training_args = TrainingArguments(
    output_dir="./modelo_gpt2",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    logging_dir="./logs_gpt2",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    weight_decay=0.01,
    learning_rate=1.5e-5
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}


trainer = Trainer(
    model=model_gpt2,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

In [ ]:
model_gpt2.save_pretrained("./gpt2_model")
tokenizer_gpt2.save_pretrained("./gpt2_model")
print("GPT-2 model and tokenizer saved in './gpt2_model'")

In [ ]:
#Modelling Distilbert

train_dataset = Dataset.from_pandas(df_train[['description_clean', 'label']])
test_dataset = Dataset.from_pandas(df_test[['description_clean', 'label']])

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize_function(example):
    return tokenizer(example['description_clean'], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.remove_columns(["description_clean"])
train_dataset = train_dataset.rename_column("label", "labels")

test_dataset = test_dataset.remove_columns(["description_clean"])
test_dataset = test_dataset.rename_column("label", "labels")

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(label_encoder.classes_)
)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)


trainer.train()

In [ ]:
test_dataset = Dataset.from_pandas(df_test[['description_clean', 'label']])

tokenized_test = test_dataset.map(tokenize_function, batched=True)

tokenized_test = tokenized_test.remove_columns(["description_clean"])

tokenized_test = tokenized_test.rename_column("label", "labels")

tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
model.save_pretrained("./distilbert_model")
tokenizer.save_pretrained("./distilbert_model")
print("DistilBERT model and tokenizer saved in './distilbert_model'")

In [ ]:
#Modelling com ModernBert

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_train['description_clean'].tolist(),
    df_train['label'].tolist(),
    test_size=0.1,
    random_state=42,
    stratify=df_train['label']
)

train_ds = Dataset.from_dict({'text': train_texts, 'label': train_labels})
val_ds   = Dataset.from_dict({'text': val_texts,   'label': val_labels})

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

train_ds = train_ds.map(tokenize_fn, batched=True)
val_ds   = val_ds.map(tokenize_fn, batched=True)

train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_ds.set_format(type="torch",   columns=["input_ids", "attention_mask", "label"])


num_labels = df_train['label'].nunique()
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)


training_args = TrainingArguments(
    output_dir="./modelo",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)


accuracy = load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return accuracy.compute(predictions=preds, references=labels)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
def tokenize_function(example):
    return tokenizer(example["description_clean"], truncation=True, padding="max_length")

test_dataset = Dataset.from_pandas(df_test[['description_clean', 'label']])

tokenized_test = test_dataset.map(tokenize_function, batched=True)

tokenized_test = tokenized_test.remove_columns(["description_clean"])

tokenized_test = tokenized_test.rename_column("label", "labels")

tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
trainer.save_model("./modernbert_model")
tokenizer.save_pretrained("./modernbert_model")
print("ModernBert model and tokenizer saved in './modernbert_model'")

#Evaluation


In [ ]:
# --- Evaluation for Logistic Regression ---
print("--- Evaluating Logistic Regression ---")
try:
    loaded_lr_model = joblib.load('logistic_regression_model.pkl')

    X_test_lr = vectorizer.transform(df_test['description_clean'])

    y_pred_lr = loaded_lr_model.predict(X_test_lr)
    print(classification_report(df_test['label'], y_pred_lr, target_names=label_encoder.classes_))
except FileNotFoundError:
    print("Logistic Regression model file not found.")
except NameError:
    print("vectorizer, df_test, or label_encoder not found. Please run the previous data preparation and modeling cells.")

# --- Evaluation for Naive Bayes ---
print("\n--- Evaluating Naive Bayes ---")
try:
    loaded_nb_pipeline = joblib.load('naive_bayes_pipeline.pkl')
    y_pred_nb = loaded_nb_pipeline.predict(df_test['description_clean'])
    print(classification_report(df_test['label'], y_pred_nb, target_names=label_encoder.classes_))
except FileNotFoundError:
    print("Naive Bayes pipeline file not found.")
except NameError:
    print("df_test or label_encoder not found. Please run the previous data preparation and modeling cells.")


# --- Evaluation for GPT-2 ---
print("\n--- Evaluating GPT-2 ---")
try:
    loaded_gpt2_model = AutoModelForSequenceClassification.from_pretrained("./gpt2_model")
    loaded_gpt2_tokenizer = AutoTokenizer.from_pretrained("./gpt2_model")
    loaded_gpt2_model.eval()

    gpt2_test_loader = DataLoader(tokenized_test, batch_size=16, collate_fn=data_collator)

    all_preds_gpt2 = []
    all_labels_gpt2 = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loaded_gpt2_model.to(device)

    with torch.no_grad():
        for batch in gpt2_test_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
            labels = batch["labels"].to(device)

            outputs = loaded_gpt2_model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            all_preds_gpt2.extend(preds.cpu().numpy())
            all_labels_gpt2.extend(labels.cpu().numpy())

    print(classification_report(all_labels_gpt2, all_preds_gpt2, target_names=label_encoder.classes_))
except FileNotFoundError:
    print("GPT-2 model files not found.")
except NameError:
     print("tokenized_test, data_collator, or label_encoder not found. Please run the previous data preparation and modeling cells.")


# --- Evaluation for DistilBERT ---
print("\n--- Evaluating DistilBERT ---")
try:
    loaded_distilbert_model = AutoModelForSequenceClassification.from_pretrained("./distilbert_model")
    loaded_distilbert_tokenizer = AutoTokenizer.from_pretrained("./distilbert_model")
    loaded_distilbert_model.eval()


    distilbert_test_loader = DataLoader(tokenized_test, batch_size=16, collate_fn=data_collator)

    all_preds_distilbert = []
    all_labels_distilbert = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loaded_distilbert_model.to(device)

    with torch.no_grad():
        for batch in distilbert_test_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
            labels = batch["labels"].to(device)

            outputs = loaded_distilbert_model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            all_preds_distilbert.extend(preds.cpu().numpy())
            all_labels_distilbert.extend(labels.cpu().numpy())

    print(classification_report(all_labels_distilbert, all_preds_distilbert, target_names=label_encoder.classes_))
except FileNotFoundError:
    print("DistilBERT model files not found.")
except NameError:
     print("tokenized_test, data_collator, or label_encoder not found. Please run the previous data preparation and modeling cells.")


# --- Evaluation for ModernBert ---
print("\n--- Evaluating ModernBert ---")
try:
    loaded_modernbert_model = AutoModelForSequenceClassification.from_pretrained("./modernbert_model")
    loaded_modernbert_tokenizer = AutoTokenizer.from_pretrained("./modernbert_model")
    loaded_modernbert_model.eval()


    modernbert_test_loader = DataLoader(tokenized_test, batch_size=16, collate_fn=data_collator)

    all_preds_modernbert = []
    all_labels_modernbert = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loaded_modernbert_model.to(device)

    with torch.no_grad():
        for batch in modernbert_test_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
            labels = batch["labels"].to(device)

            outputs = loaded_modernbert_model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            all_preds_modernbert.extend(preds.cpu().numpy())
            all_labels_modernbert.extend(labels.cpu().numpy())

    print(classification_report(all_labels_modernbert, all_preds_modernbert, target_names=label_encoder.classes_))
except FileNotFoundError:
    print("ModernBert model files not found.")
except NameError:
     print("tokenized_test, data_collator, or label_encoder not found. Please run the previous data preparation and modeling cells.")
except Exception as e:
    print(f"An error occurred during ModernBert evaluation: {e}")

#Deployment

In [ ]:
!ngrok config add-authtoken 2uDsyoIapgKUCH1o6q9vLuhK6Wy_49xGqyHvp94UYF13n9BoH

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./modernbert_model")
model = AutoModelForSequenceClassification.from_pretrained("./modernbert_model")
model.eval()

label2cat = {
    0: "Cardiovascular / Pulmonary",
    1: "ENT - Otolaryngology",
    2: "Gastroenterology",
    3: "Hematology - Oncology",
    4: "Nephrology",
    5: "Neurology",
    6: "Neurosurgery",
    7: "Obstetrics / Gynecology",
    8: "Ophthalmology",
    9: "Orthopedic",
    10: "Pediatrics - Neonatal",
    11: "Psychiatry / Psychology",
    12: "Radiology"
}

st.title("🧠 Classificador Médico por Descrição do Paciente")
st.write("Insira abaixo uma descrição do paciente para prever a especialidade médica correspondente.")

input_text = st.text_area("📋 Descrição do paciente:")

if st.button("🔍 Classificar"):
    if input_text.strip() == "":
        st.warning("Por favor, digite uma descrição válida.")
    else:
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)
            predicted_label = torch.argmax(probs, dim=1).item()
            confidence = probs[0][predicted_label].item()

        st.success(f"🩺 Especialidade prevista: **{label2cat[predicted_label]}**")
        st.info(f"🔢 Confiança: {confidence:.2%}")

In [ ]:
code = '''
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("./modernbert_model")
model = AutoModelForSequenceClassification.from_pretrained("./modernbert_model")
model.eval()

label2cat = {
    0: "Cardiovascular / Pulmonary",
    1: "ENT - Otolaryngology",
    2: "Gastroenterology",
    3: "Hematology - Oncology",
    4: "Nephrology",
    5: "Neurology",
    6: "Neurosurgery",
    7: "Obstetrics / Gynecology",
    8: "Ophthalmology",
    9: "Orthopedic",
    10: "Pediatrics - Neonatal",
    11: "Psychiatry / Psychology",
    12: "Radiology"
}

st.title("🧠 Classificador Médico por Descrição do Paciente")
st.write("Insira abaixo uma descrição do paciente para prever a especialidade médica correspondente.")

input_text = st.text_area("📋 Descrição do paciente:")

if st.button("🔍 Classificar"):
    if input_text.strip() == "":
        st.warning("Por favor, digite uma descrição válida.")
    else:
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)
            predicted_label = torch.argmax(probs, dim=1).item()
            confidence = probs[0][predicted_label].item()

        st.success(f"🩺 Especialidade prevista: **{label2cat[predicted_label]}**")
        st.info(f"🔢 Confiança: {confidence:.2%}")
'''

with open("app.py", "w") as f:
    f.write(code)


In [ ]:
!ls app.py


In [ ]:
!pkill streamlit || true
!streamlit run app.py &> streamlit.log &


In [ ]:
from pyngrok import ngrok
import time

# Aguarda o app iniciar
time.sleep(5)

# Cria o túnel na porta 8501
public_url = ngrok.connect(8501)
print(f"✅ App disponível em: {public_url}")


In [ ]:
import streamlit as st
import torch
from transformers import autotokenizer, automodelforsequenceclassification